In [1]:
import yfinance as yf
import ta
import pandas as pd
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
import numpy as np
import hvplot.pandas

/Users/DinoK1/anaconda3/envs/dev/lib/python3.10/site-packages/backtesting/_plotting.py:50: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

In [2]:
class pricecross(Strategy):

    n1 = 2

    def init(self):
        close = self.data.Close
        self.sma = self.I(ta.trend.sma_indicator, pd.Series(close), self.n1)

    def next(self):
        if crossover(df['Close'], self.sma):
            self.buy()
        elif crossover(self.sma, df['Close']):
            self.sell()

In [3]:
df = yf.download('ETH-USD', interval='1m', period='7D')

[*********************100%%**********************]  1 of 1 completed


In [4]:
df.head()

,Open,High,Low,Close,Adj Close,Volume
Datetime,,,,,,
2024-01-03 00:00:00+00:00,2355.981445,2355.981445,2355.981445,2355.981445,2355.981445,0
2024-01-03 00:01:00+00:00,2356.685059,2356.685059,2356.685059,2356.685059,2356.685059,0
2024-01-03 00:03:00+00:00,2358.718994,2358.718994,2358.718994,2358.718994,2358.718994,0
2024-01-03 00:04:00+00:00,2359.686523,2359.686523,2359.686523,2359.686523,2359.686523,0
2024-01-03 00:05:00+00:00,2358.689209,2358.689209,2358.689209,2358.689209,2358.689209,0


In [5]:
bt = Backtest(df, pricecross, cash=100000, commission=0.00,
exclusive_orders=True)

output = bt.run()

In [6]:
output

Start                     2024-01-03 00:00...
End                       2024-01-09 01:31...
Duration                      6 days 01:31:00
Exposure Time [%]                   91.736414
Equity Final [$]                103863.765137
Equity Peak [$]                 103967.970703
Return [%]                           3.863765
Buy & Hold Return [%]               -2.215096
Return (Ann.) [%]                 2128.231164
Volatility (Ann.) [%]              834.045336
Sharpe Ratio                         2.551697
Sortino Ratio                      189.131448
Calmar Ratio                       330.521662
Max. Drawdown [%]                   -6.439007
Avg. Drawdown [%]                   -1.159971
Max. Drawdown Duration        3 days 22:30:00
Avg. Drawdown Duration        0 days 08:52:00
# Trades                                    8
Win Rate [%]                             87.5
Best Trade [%]                       2.110598
Worst Trade [%]                     -0.016127
Avg. Trade [%]                    

In [7]:
optim = bt.optimize(n1 = range(2,6,4),
    maximize= 'Return [%]')

/Users/DinoK1/anaconda3/envs/dev/lib/python3.10/site-packages/backtesting/backtesting.py:1375: UserWarning: For multiprocessing support in `Backtest.optimize()` set multiprocessing start method to 'fork'.
  warnings.warn("For multiprocessing support in `Backtest.optimize()` "


  0%|          | 0/1 [00:00<?, ?it/s]

In [8]:
optim

Start                     2024-01-03 00:00...
End                       2024-01-09 01:31...
Duration                      6 days 01:31:00
Exposure Time [%]                   91.736414
Equity Final [$]                103863.765137
Equity Peak [$]                 103967.970703
Return [%]                           3.863765
Buy & Hold Return [%]               -2.215096
Return (Ann.) [%]                 2128.231164
Volatility (Ann.) [%]              834.045336
Sharpe Ratio                         2.551697
Sortino Ratio                      189.131448
Calmar Ratio                       330.521662
Max. Drawdown [%]                   -6.439007
Avg. Drawdown [%]                   -1.159971
Max. Drawdown Duration        3 days 22:30:00
Avg. Drawdown Duration        0 days 08:52:00
# Trades                                    8
Win Rate [%]                             87.5
Best Trade [%]                       2.110598
Worst Trade [%]                     -0.016127
Avg. Trade [%]                    

In [9]:
#build interactive backtest graph

signals_df = df

window = 2

signals_df.head()

,Open,High,Low,Close,Adj Close,Volume
Datetime,,,,,,
2024-01-03 00:00:00+00:00,2355.981445,2355.981445,2355.981445,2355.981445,2355.981445,0
2024-01-03 00:01:00+00:00,2356.685059,2356.685059,2356.685059,2356.685059,2356.685059,0
2024-01-03 00:03:00+00:00,2358.718994,2358.718994,2358.718994,2358.718994,2358.718994,0
2024-01-03 00:04:00+00:00,2359.686523,2359.686523,2359.686523,2359.686523,2359.686523,0
2024-01-03 00:05:00+00:00,2358.689209,2358.689209,2358.689209,2358.689209,2358.689209,0


In [10]:
signals_df = df.loc[:,["Close"]]

In [11]:
window = 2

In [12]:
signals_df['MA'] = signals_df['Close'].rolling(window=window).mean()

signals_df['Signal'] = 0.0


signals_df['Signal'][window:] = np.where(
    signals_df['Close'][window:] > signals_df['MA'][window:], 1.0, 0.0
)

In [13]:
signals_df['Entry/Exit'] = signals_df['Signal'].diff()

signals_df.tail(10)

,Close,MA,Signal,Entry/Exit
Datetime,,,,
2024-01-09 01:22:00+00:00,2312.018799,2312.502808,0.0,-1.0
2024-01-09 01:23:00+00:00,2309.015381,2310.517090,0.0,0.0
2024-01-09 01:24:00+00:00,2307.117188,2308.066284,0.0,0.0
2024-01-09 01:25:00+00:00,2306.337158,2306.727173,0.0,0.0
2024-01-09 01:26:00+00:00,2305.511230,2305.924194,0.0,0.0
2024-01-09 01:27:00+00:00,2305.334473,2305.422852,0.0,0.0
2024-01-09 01:28:00+00:00,2304.557373,2304.945923,0.0,0.0
2024-01-09 01:29:00+00:00,2302.177490,2303.367432,0.0,0.0
2024-01-09 01:30:00+00:00,2304.165771,2303.171631,1.0,1.0


In [14]:
exit = signals_df[signals_df['Entry/Exit'] == -1.0]['Close'].hvplot.scatter(
    color='yellow',
    marker='v',
    size=200,
    legend=False,
    ylabel='Price in $',
    width=1000,
    height=400
)

entry = signals_df[signals_df['Entry/Exit'] == 1.0]['Close'].hvplot.scatter(
    color='purple',
    marker='^',
    size=200,
    legend=False,
    ylabel='Price in $',
    width=1000,
    height=400
)

security_close = signals_df[['Close']].hvplot(
    line_color='blue',
    ylabel='Price in $',
    width=1000,
    height=400
)


moving_avgs = signals_df[['MA']].hvplot(
    ylabel='Price in $',
    width=1000,
    height=400
)


entry_exit_plot = security_close * moving_avgs * entry * exit

entry_exit_plot.opts(
    title="ETH - MA, Entry and Exit Points"
)

:Overlay
   .Curve.I          :Curve   [Datetime]   (Close)
   .Curve.II         :Curve   [Datetime]   (MA)
   .Scatter.Close.I  :Scatter   [Datetime]   (Close)
   .Scatter.Close.II :Scatter   [Datetime]   (Close)

In [15]:
# Build performance table
initial_capital = float(100000)
share_size = 42
# columns
signals_df['Position'] = share_size * signals_df['Signal']
signals_df['Entry/Exit Position'] = signals_df['Position'].diff()
signals_df['Portfolio Holdings'] = signals_df['Close'] * signals_df['Position']
signals_df['Portfolio Cash'] = initial_capital - (signals_df['Close'] * signals_df['Entry/Exit Position']).cumsum() 
signals_df['Portfolio Total'] = signals_df['Portfolio Cash'] + signals_df['Portfolio Holdings']
signals_df['Portfolio Daily Returns'] = signals_df['Portfolio Total'].pct_change()
signals_df['buy and hold'] = signals_df['Close'] * 42
signals_df.tail()

,Close,MA,Signal,Entry/Exit,Position,Entry/Exit Position,Portfolio Holdings,Portfolio Cash,Portfolio Total,Portfolio Daily Returns,buy and hold
Datetime,,,,,,,,,,,
2024-01-09 01:27:00+00:00,2305.334473,2305.422852,0.0,0.0,0.0,0.0,0.000000,122598.378906,122598.378906,0.000000,96824.047852
2024-01-09 01:28:00+00:00,2304.557373,2304.945923,0.0,0.0,0.0,0.0,0.000000,122598.378906,122598.378906,0.000000,96791.409668
2024-01-09 01:29:00+00:00,2302.177490,2303.367432,0.0,0.0,0.0,0.0,0.000000,122598.378906,122598.378906,0.000000,96691.454590
2024-01-09 01:30:00+00:00,2304.165771,2303.171631,1.0,1.0,42.0,42.0,96774.962402,25823.416504,122598.378906,0.000000,96774.962402
2024-01-09 01:31:00+00:00,2303.794189,2303.979980,0.0,-1.0,0.0,-42.0,0.000000,122582.772461,122582.772461,-0.000127,96759.355957


In [16]:
# build graph
exit = signals_df[signals_df['Entry/Exit'] == -1.0]['Portfolio Total'].hvplot.scatter(
    color='yellow',
    marker='v',
    legend=False,
    ylabel='Total Portfolio Value',
    width=1000,
    height=400
)

entry = signals_df[signals_df['Entry/Exit'] == 1.0]['Portfolio Total'].hvplot.scatter(
    color='purple',
    marker='^',
    ylabel='Total Portfolio Value',
    width=1000,
    height=400
)

total_portfolio_value = signals_df[['Portfolio Total']].hvplot(
    line_color='lightgray',
    ylabel='Total Portfolio Value',
    xlabel='Date',
    width=1000,
    height=400
)

portfolio_entry_exit_plot = total_portfolio_value * entry * exit
portfolio_entry_exit_plot.opts(
    title="ETH Algorithm - Total Portfolio Value",
    yformatter='%.0f'
)

:Overlay
   .Curve.I                    :Curve   [Datetime]   (Portfolio Total)
   .Scatter.Portfolio_Total.I  :Scatter   [Datetime]   (Portfolio Total)
   .Scatter.Portfolio_Total.II :Scatter   [Datetime]   (Portfolio Total)

In [17]:
portfolio_total = signals_df['Portfolio Total'].hvplot(
    line_color='gray',
    ylabel='Total Portfolio Value',
    xlabel='Date',
    width=1000,
    height=400
)
closing = signals_df['buy and hold'].hvplot(
    line_color='purple',
    ylabel='Total Portfolio Value',
    width=1000,
    height=400
)

performance = portfolio_total * closing

performance.opts(
    title="ETH Algorithm - Total Portfolio Value",
    yformatter='%.0f'
)

:Overlay
   .Curve.Portfolio_Total :Curve   [Datetime]   (Portfolio Total)
   .Curve.Buy_and_hold    :Curve   [Datetime]   (buy and hold)

In [18]:
signals_df.describe()

,Close,MA,Signal,Entry/Exit,Position,Entry/Exit Position,Portfolio Holdings,Portfolio Cash,Portfolio Total,Portfolio Daily Returns,buy and hold
count,8483.000000,8482.000000,8483.000000,8482.000000,8483.000000,8482.000000,8483.000000,8482.000000,8482.000000,8481.000000,8483.000000
mean,2255.645126,2255.636373,0.498762,0.000000,20.948014,0.000000,47261.773818,60993.595795,108260.941621,0.000024,94737.095272
std,44.546609,44.528186,0.500028,0.678471,21.001174,28.495787,47400.471984,47740.829189,6379.673703,0.000523,1870.957561
min,2113.925293,2120.409302,0.000000,-1.000000,0.000000,-42.000000,0.000000,242.442871,96928.965332,-0.024207,88784.862305
25%,2231.492554,2231.546417,0.000000,0.000000,0.000000,0.000000,0.000000,13924.678589,102450.406738,0.000000,93722.687256
50%,2243.716309,2243.784790,0.000000,0.000000,0.000000,0.000000,0.000000,97677.387695,107898.450684,0.000000,94236.084961
75%,2263.918823,2263.858002,1.000000,0.000000,42.000000,0.000000,94238.735596,107854.330688,112660.754395,0.000029,95084.590576
max,2385.117676,2384.310791,1.000000,1.000000,42.000000,42.000000,100174.942383,122639.035645,122648.264160,0.011577,100174.942383
